In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('house_data/train.csv')
test_df = pd.read_csv('house_data/test.csv')

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [4]:
set(train_df.columns) - set(test_df.columns)

{'SalePrice'}

In [5]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [10]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
import xgboost as xgb

In [11]:
train, test = train_test_split(train_df, test_size=.2, random_state=0)

In [14]:
train_target = train.SalePrice.values
train.drop(['Id', 'SalePrice'], axis=1, inplace=True)
test_target = test.SalePrice.values
test.drop(['Id', 'SalePrice'], axis=1, inplace=True)

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [36]:
skf = KFold(n_splits=3, random_state=0)
for train_index, test_index in skf.split(train):
    X_train = train.iloc[train_index]
    X_test = train.iloc[test_index]
    y_train = train_target[train_index]
    y_test = train_target[test_index]
    
    # mdl = Ridge(random_state=0)
    gs = GridSearchCV(Ridge(), 
                      {'alpha': [0., .01, .1, .5, 1., 10., 100., 1000.],
                       'fit_intercept': [True, False],
                       'normalize': [True, False]},
                      scoring='neg_mean_squared_error')
    numerics = X_train.loc[:, X_train.dtypes != np.object]
    numerics_means = numerics.mean(axis=0)
    numerics_filled = numerics.fillna(numerics.mean(axis=0))
    
    gs.fit(numerics_filled, y_train)
    print(gs.best_params_)
    mdl = Ridge(random_state=0, **gs.best_params_)
    mdl.fit(numerics_filled, y_train)
    preds = mdl.predict(X_test.loc[:, X_test.dtypes != np.object].fillna(numerics_means))
    print(np.mean(np.sqrt((preds - y_test)**2)))

/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.0223345758142504e-17 / 1.1102230246251565e-16
  RuntimeWarning)


{'alpha': 0.0, 'fit_intercept': False, 'normalize': True}
21498.4649454


/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 7.662134039217025e-20 / 1.1102230246251565e-16
  RuntimeWarning)


{'alpha': 0.1, 'fit_intercept': True, 'normalize': True}
22955.4466738
{'alpha': 0.1, 'fit_intercept': True, 'normalize': True}
19482.2506772


/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 3.7979488038421506e-19 / 1.1102230246251565e-16
  RuntimeWarning)
/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 4.417769450622814e-19 / 1.1102230246251565e-16
  RuntimeWarning)


In [40]:
skf = KFold(n_splits=3, random_state=0)
for train_index, test_index in skf.split(train):
    X_train = train.iloc[train_index]
    X_test = train.iloc[test_index]
    y_train = train_target[train_index]
    y_test = train_target[test_index]
    
    # mdl = Ridge(random_state=0)
    gs = GridSearchCV(GradientBoostingRegressor(), 
                      {'learning_rate': [.01, .1, .5, 1.],
                       'n_estimators': list(range(2, 50, 10)),
                       'subsample': [.1, .3, .5, .7, .8, .9, 1.],
                       'max_depth': [2, 3, 4, 5, 8, 10, 12, 15, 20]},
                      scoring='neg_mean_squared_error')
    numerics = X_train.loc[:, X_train.dtypes != np.object]
    numerics_means = numerics.mean(axis=0)
    numerics_filled = numerics.fillna(numerics.mean(axis=0))
    
    gs.fit(numerics_filled, y_train)
    print(gs.best_params_)
    mdl = GradientBoostingRegressor(random_state=0, **gs.best_params_)
    mdl.fit(numerics_filled, y_train)
    preds = mdl.predict(X_test.loc[:, X_test.dtypes != np.object].fillna(numerics_means))
    print(np.mean(np.sqrt((preds - y_test)**2)))

{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 42, 'subsample': 0.3}
18182.8283775
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 32, 'subsample': 0.3}
20335.6010282
{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 32, 'subsample': 0.3}
17465.2843852
